# **DIPLOMATURA UNIVERSITARIA EN CIENCIA DE DATOS**

# Módulo 5: Aprendizaje No Supervisado

## REDUCCIÓN DE DIMENSIONALIDAD: Análisis en componentes principales (PCA).

## **Docentes:** Lic. Luis Duarte - Dra. Griselda Bobeda  - Dra. Magdalena Lucini

### Octubre 2024, FaCENA - UNNE

### Contacto:  
* luis.duarte@comunidad.unne.edu.ar;
* griseldabobeda@gmail.com;
* mmlucini@comunidad.unne.edu.ar

**Objetivos:**

*   Aplicar diferentes técnicas de reducción de dimensionalidad a situaciones con datos reales.
*   Realizar análisis gráfico de resultados.
*   Producir estimaciones a partir de estas técnicas.


**Antes de empezar:**

Necesitamos importar las siguientes librerías:


## Primera parte: Construcción de las Componentes Principales


### 1.1) Importe la base de datos.

In [ ]:
!pip install basemap
import numpy as np
%matplotlib inline
%pylab inline
from matplotlib import pyplot as plt
import pandas as pd
import io
import seaborn as sns
from mpl_toolkits.basemap import Basemap
from google.colab import files

Populating the interactive namespace from numpy and matplotlib


In [ ]:
uploaded = files.upload()

Saving Country-data.csv to Country-data.csv


In [ ]:
paises = pd.read_csv(io.BytesIO(uploaded['Country-data (2).csv']))

KeyError: 'Country-data (2).csv'

### 1.2) Realice la exploración de los datos.


In [ ]:
paises.head()

NameError: name 'paises' is not defined

In [ ]:
#Veamos la dimensión de la base:
paises.shape

NameError: name 'paises' is not defined

La base cuenta con 167 países y 9 variables socioeconómicas.

La base de datos "paises.csv" corresponde a 167 países del mundo, junto a variables socio-económicas de los mismos. Fue extraída de https://www.kaggle.com/datasets/vipulgohel/clustering-pca-assignment/data .


* `country`: Nombre del país.
* `child_mort`: Muertes de niños menores de 5 años por cada 1000 nacidos vivos.
* `exports`: Exportaciones de bienes y servicios. Dado como porcentaje del PIB total.
* `health`: Gasto total en salud como porcentaje del PIB total.
* `imports`: Importaciones de bienes y servicios. Dado como porcentaje del PIB total.
* `Income`: Ingreso neto por persona.
* `Inflation`: Medición de la tasa de crecimiento anual del PIB total.
* `life_expec`: Promedio de años que viviría un recién nacido si se mantienen los patrones actuales de mortalidad.
* `total_fer`: Número de hijos que nacerían por mujer si se mantienen las tasas actuales de fertilidad por edad.
* `gdpp`: PIB per cápita. Calculado como el PIB total dividido por la población total.

In [ ]:
# veamos una descripción unidimensional de los datos
paises.describe()

In [ ]:
# Podemos chequear las presencia de valores faltantes:
paises.isnull().sum()

In [ ]:
#Veamos el tipo de datos que tiene nuestra base:
paises.dtypes

In [ ]:
paises1=paises.drop('country', axis=1, inplace=False)


In [ ]:
paises1.head()

In [ ]:
plt.figure(figsize = (20,10))
g=paises1.corr()
b=sns.heatmap(g,annot = True)
plt.setp(b.get_xticklabels(), rotation=90)
plt.setp(b.get_yticklabels(), rotation=0)
plt.yticks(fontsize=14, fontweight='bold')
plt.xticks(fontsize=14, fontweight='bold')
plt.show();

### 1.3) Calcule las componentes principales basados en la base pasises1.csv.

Antes de calcular las Componentes Principales, es necesario decidir si es necesario realizar un preprocesamiento respecto al escalamiento. ¿Qué opinan?

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scaler = StandardScaler()
scaled_paises1 = scaler.fit_transform(paises1)
pca = PCA()
pca.fit(scaled_paises1)

In [ ]:
#Varianza Explicada por las componentes
pca.explained_variance_

In [ ]:
# Proporción de Varianza Explicada
PVE=pca.explained_variance_ratio_
PVE

In [ ]:
# Proporción Acumulada de Varianza Explicada
np.cumsum(PVE)

## ¿Cuántas componentes seleccionamos?
### Algunos criterios:

*   **Gráfico de sedimentación:** elegir las componentes (ordenadas de mayor a menor) hasta donde la pendiente comienza a nivelarse.
*   Seleccionar las componentes hasta cubrir un porcentaje determinado de la varianza.
*   **Establecer una cota mínima**; por ejemplo la varianza media. En el caso de trabajar con las variables estandarizadas: $\lambda \geq 1$.

In [ ]:
# Trazo del gráfico de sedimentación (proporción de varianza explicada acumulada.)
fig = plt.figure(figsize = (12,5))
plt.plot(np.arange(1,10),np.cumsum(pca.explained_variance_ratio_))
plt.title('Gráfico de Sedimentación', fontsize=14)
plt.xlabel('Número de Componentes', fontsize=12, fontweight='bold')
plt.ylabel('Proporción de Varianza Explicada Acumulada', fontsize=12,fontweight='bold')
plt.yticks(fontsize=12, fontweight='bold')
plt.xticks(fontsize=12, fontweight='bold')
plt.show()

In [ ]:
#Nos quedamos con las primeras 4 componentes
pca = PCA(n_components=4)
pca.fit(scaled_paises1)
PCAs2=pca.components_

Veamos las cargas de las variables en cada una de las componentes.

In [ ]:
colnames = list(paises1.columns)
prinComp_df = pd.DataFrame({ 'Variables':colnames,'PC1':pca.components_[0],'PC2':pca.components_[1],'PC3':pca.components_[2],'PC4':pca.components_[3]})
prinComp_df

* Graficamos las cargas de las variables en cada componente.

In [ ]:
loadings_df = prinComp_df.set_index('Variables')

# Graficar el mapa de calor con seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(loadings_df, annot=True, cmap='coolwarm', center=0)
plt.title('Mapa de calor de las cargas del PCA')
plt.xlabel('Componentes principales')
plt.ylabel('Variables')
plt.show()

In [ ]:
for i in range(4):
  plt.figure(figsize=(12, 6))
#component_number = 0  # 0 para la primera componente, 1 para la segunda, etc.
  plt.bar(range(PCAs2.shape[1]), PCAs2[i, :])
  plt.xlabel('Variables originales')
  plt.ylabel('Peso en la Componente Principal')
  plt.title(f'Cargas de las variables en la Componente Principal {i + 1}')
  plt.xticks(range(PCAs2.shape[1]), paises1.columns.tolist())
  plt.grid(axis='y')
  plt.tight_layout()

plt.show()

* Veamos los datos proyectados en el nuevo espacio generado por las 4 primeras componentes.

In [ ]:
import plotly.graph_objs as go
reduced_data = pca.fit_transform(scaled_paises1)

In [ ]:
graficos = go.Scatter(x=reduced_data[:,0], y=reduced_data[:,1],
                           mode='markers',
                           text=paises['country'].tolist())
layout = go.Layout(title="Proyección de los Países",titlefont=dict(size=20),
                xaxis=dict(title='CP1'),
                yaxis=dict(title='CP2'),
                autosize=False, width=1000,height=650)

fig = go.Figure(data=graficos, layout=layout)
fig.show(renderer="colab")

Veamos la matriz de correlación de los datos proyectados. Esperaríamos que fuera...

In [ ]:
corrmat = np.corrcoef(reduced_data.T)
plt.figure(figsize = (8,5))
sns.heatmap(corrmat,cmap="YlGnBu",annot = True)
plt.setp(b.get_xticklabels(), fontweight='bold')
plt.yticks(fontsize=14, fontweight='bold')
plt.xticks(fontsize=14, fontweight='bold')
plt.show()

## Segunda parte: Aplicación de reducción de dimensiones a datos climáticos.

### 2.1) Importe el segundo conjunto de datos: SST.csv, el cual corresponde a las SST promedios (en Enero) en el golfo de México, para cada año entre 1998 y 2015. ¿Cuáles son las filas y columnas de este nuevo conjunto de datos?

In [ ]:
#########
# TO DO #
#########
#from google.colab import files
uploaded = files.upload()
SST = pd.read_csv(io.BytesIO(uploaded['SST.csv']))
# Hechemos un vistazo a la tabla
SST

### 2.2) Usando las funciones *corrcoef* y *pcolor*, calcule y grafique la matriz de correlaciones entre los 1595 píxeles de SST.

In [ ]:
#########
# TO DO #
#########
# calculamos la matriz de correlaciones
corr_sst = corrcoef(SST.T)
# plot the correlation matrix
pcolor(corr_sst, cmap='bwr', vmin=-1, vmax=1)
title('Correlación de la SST en el Golfo de México', size=20)
colorbar()

### 2.3) Luego, representamos gráficamente el mapa de correlación entre la SST en el píxel número 1300 y los otros 1594 píxeles. También mostramos la ubicación del píxel número 1300. Necesitaremos cargar la base de datos 'data1.csv' que contiene las coordenadas de los píxeles.

In [ ]:
#########
# TO DO #
#########
# Importamos 'data1.csv'
uploaded = files.upload()
data1=pd.read_csv(io.BytesIO(uploaded['data1.csv']))

In [ ]:
#Definimos una función que nos permita graficar:
# function to plot images
def plot_im(lon, lat, im, size_points, var_name, cmap, vmin, vmax):

    # transform to arrays (just in case)
    lon = array(lon)
    lat = array(lat)
    im = array(im)

    # Mercator projection (for small zone)
    m = Basemap(projection='merc', llcrnrlat=nanmin(lat), urcrnrlat=nanmax(lat),\
                llcrnrlon=nanmin(lon), urcrnrlon=nanmax(lon), lat_0=(nanmax(lat)+nanmin(lat))*0.5,\
                lon_0=(nanmax(lon)+nanmin(lon))*0.5, resolution='l')
    # you can use other projections (see https://matplotlib.org/basemap/users/mapsetup.html)

    # transform (lon,lat) to (x,y)
    x,y = m(lon,lat)

    # plot
    im = ma.masked_where(isnan(im), im)
    res = m.scatter(x, y, size_points, im, 'o', alpha=1, cmap=cmap, lw=0, vmin=vmin, vmax=vmax)
    m.drawcoastlines()
    m.fillcontinents()
    parallels = linspace(ceil(nanmin(lat)), floor(nanmax(lat)), 1+int(floor(nanmax(lat))-ceil(nanmin(lat))))
    meridians = linspace(ceil(nanmin(lon)), floor(nanmax(lon)), 1+int(floor(nanmax(lon))-ceil(nanmin(lon))))
    m.drawparallels(parallels, labels=[1,0,0,1], fontsize=10)
    m.drawmeridians(meridians, labels=[1,0,0,1], fontsize=10)
    cb = m.colorbar(res, location="right")
    cb.set_label(var_name, fontsize=20)

In [ ]:
# graficamos la correlación entre un píxel y los demás.
plt.figure(figsize=(15, 6))
plot_im(data1.lon, data1.lat, corr_sst[1299,:], 70, 'Correlación', cmap='bwr', vmin=-1, vmax=1)
title('Correlación entre la SST del píxel 1300 y las demás', size=20)

# print las coordenadas del píxel de interés
print('Longitud: ', str(360-data1.lon[1300]))
print('Latitud: ', str(data1.lat[1300]))

**Tarea:** Realice el mismo análisis para los píxeles 100 y 1000.

In [ ]:
#########
# TO DO #
#########


### 2.4) Calcule las primeros 10 PCA de la SST en el Golfo de México y trace el gráfico de sedimentación para la varianza explicada acumulada. ¿Cuántas componentes serían necesarias para explicar la variabilidad de la SST? (Hacer)

In [ ]:
#############
### TO DO ###
#############
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

SST_PCA = PCA(n_components=10)
SST_PCA.fit(SST)
plot(np.arange(1,11), cumsum(SST_PCA.explained_variance_ratio_), 'b-*')
xlabel('Número de Componentes Principales', size=10)
ylabel('Porcentaje de varianza explicada acumulado', size=10)

### 2.5) Trace las CPs que considere, almacenadas en SST_PCA.components_, utilizando la función *plot_im*. ¿Qué destacan las CPs?

In [ ]:
#############
### TO DO ###
#############
# Graficamos cada componente principal.
for k in range(4):
  figure()
  plt.figure(figsize=(15, 6))
  plot_im(data1.lon, data1.lat, SST_PCA.components_[k,:],70, 'Cargas',
        cmap='jet', vmin=min(SST_PCA.components_[k,:]), vmax=max(SST_PCA.components_[k,:]))
  title('CP ' + str(k+1), size=20)

### 2.6) Para finalizar, reconstruya el mapa de SST en enero de 1998 (es decir, la primera fila del conjunto de datos) utilizando las primeras $k$ CPs consideradas en los ítems anteriores. Para ello, proyecta el conjunto de datos original en el subespacio de dimensión $k$, utilizando la función de *transform* del objeto SST_PCA. Ten en cuenta que debes agregar la media de la SST al mapa reconstruido. (Hacer)

In [ ]:
# Calculamos las proyecciones de los datos en las primeras 4 componentes
SST_proy = SST_PCA.transform(SST)
SST_est = SST_proy @ SST_PCA.components_
SST_est_con_media = SST_est + mean(SST.values, 0)

# Graficamos la SST observada en Enero de 1998
figure()
plt.figure(figsize=(15, 6))
plot_im(data1.lon, data1.lat, SST.values[0,:], 70, 'SST (°C)',
        cmap='rainbow', vmin=min(SST.values[0,:]), vmax=max(SST.values[0,:]))
title('SST observada en Enero de 1998', size=20)

# plot the approximated SST in January 1998 using 4 EOFs
figure()
plt.figure(figsize=(15, 6))
plot_im(data1.lon, data1.lat, SST_est_con_media[0,:], 70, 'SST (°C)',
        cmap='rainbow', vmin=min(SST.values[0,:]), vmax=max(SST.values[0,:]))
title('Aproximación de la SST en Enero de 1998 utilizando 4 PCs', size=20)



Calculemos el error de estimación utilizando las primeras 4 Componentes Principales para enero del año 1998.

Recordemos que el error viene dado por:

$$\frac{1}{N}\sum_{i=1}^N\|\mathbf{x}_n-\widetilde{\mathbf{x
}}_n\|$$

In [ ]:
E=1/SST.values[0,:].shape[0]*np.dot(SST_est_con_media[0,:]-SST.values[0,:],SST_est_con_media[0,:]-SST.values[0,:])
print('El error de reconstruir la variable SST para enero de 1998 utilizando 4 componentes es:', E)

### ¿Qué pasa con el error si se hace la reconstrucción para enero de 1998 utilizando 10 componentes?

In [ ]:
#########
# TO DO #
#########

## Tercera parte: Aplicación de PCA a imágenes

La siguiente base de datos contiene 213 imágenes de 7 expresiones faciales (6 expresiones faciales básicas + 1 neutra) interpretadas por 10 modelos japonesas. La misma fue planificada y ensamblada por Michael Lyons, Miyuki Kamachi y Jiro Gyoba. Las fotos fueron tomadas en el Departamento de Psicología de la Universidad de Kyushu.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

3.1) Lea la base de datos (En este cuaderno está leído directamente desde una carpeta de drive propia) y explórela.

In [ ]:
carpeta_imagenes = '/content/drive/MyDrive/DIPLOMATURA/Reducción de Dim/Bases/jaffedbase'
imagenes = []

for nombre_archivo in os.listdir(carpeta_imagenes):
    if nombre_archivo.endswith('.tiff'):
        ruta_imagen = os.path.join(carpeta_imagenes, nombre_archivo)
        imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
        imagenes.append(imagen)

imagenes = np.array(imagenes)

In [ ]:
#########
# TO DO #
#########

In [ ]:
total_imagenes = imagenes.shape[0]
indices_seleccionados = np.random.choice(total_imagenes, 10, replace=False)

plt.figure(figsize=(15, 10))

for i in np.arange(5):
    imagen1 = imagenes[np.random.choice(total_imagenes)]
    imagen2 = imagenes[np.random.choice(total_imagenes)]

    plt.subplot(5, 2, 2*i + 1)
    plt.imshow(imagen1,cmap='gray')
    #plt.title('Imagen Original')
    plt.axis('off')

    plt.subplot(5, 2, 2*i + 2)
    plt.imshow(imagen2, cmap='gray')
    plt.axis('off')

# Ajustar el diseño y mostrar todas las imágenes
plt.tight_layout()
plt.show()

In [ ]:
imagenes_mod=imagenes.reshape(imagenes.shape[0], -1)

In [ ]:
imagenes_mod.shape

(213, 65536)

3.2) Aplique un PCA con el número de componentes necesarias para explicar el 75% de la varianza.

In [ ]:
#############
### TO DO ###
#############


3.3) Proyecte los datos sobre las nuevas componentes y reconstruya alguna de las fotos de la base.

In [ ]:
#########
# TO DO #
#########

Reconstruyamos alguna imagen

In [ ]:
#########
# TO DO #
#########

In [ ]:
plt.figure(figsize=(15, 10))


# Imagen original
plt.subplot(1, 3, 1)
plt.imshow(im_original,cmap='gray')
#plt.title('Imagen Original')
plt.axis('off')

# Imagen media
plt.subplot(1, 3, 2)
plt.imshow(im_media, cmap='gray')
#plt.title('Imagen Media')
plt.axis('off')

# Imagen reconstruida
plt.subplot(1, 3, 3)
plt.imshow(im_rec, cmap='gray')
#plt.title('Imagen Reconstruida')
plt.axis('off')

# Ajustar el diseño y mostrar todas las imágenes
plt.tight_layout()
plt.show()

3.4) ¿Qué error se comete en la estimación reconstrucción de esta imagen con las componentes seleccionadas?

In [ ]:
#########
# TO DO #
#########